Splits documents, tokenizes as embeddings, and stores them in a vector store

In [1]:
# import packages
from dotenv import load_dotenv
import os
import getpass
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings

root_path = os.getcwd()
# NOTE: change files as needed
textbook_extracted_path = os.path.join(root_path,'..', 'data_processing', 'processed_data','kang_math_textbook_edited.md')
vector_path = os.path.join(root_path, 'vector-stores', 'test_vector_store')

In [2]:
# Load the organized textbook data
## CHANGE FILE HERE FOR EMBEDS
with open(textbook_extracted_path, "r", encoding="utf-8") as file:
    extracted_text = file.read()    

In [3]:
# Create a RecursiveCharacterTextSplitter object to split the text into chunks

text_splitter = RecursiveCharacterTextSplitter(
    separators=['#','##','###'],
    chunk_size=1000,       # Maximum number of characters in each chunk
    chunk_overlap=200,     # Number of characters that overlap between consecutive chunks
    length_function=len,    # Function to measure the length of chunks
)

texts = text_splitter.split_text(extracted_text)

In [4]:
texts

['# 國民小學第九冊\n---\n# 國民小學數學 教師手冊 資料篇 第九冊 5 上\n編輯要旨\n一、 本版教科書依據民國一○三年教育部發布之「十二年國民基本教育課程綱要總綱」及民國一○七年\n發布之「十二年國民基本教育數學領域課程綱要」編輯而成。\n二、 本書於國小階段共十二冊 供國民小學一至六年級數學領域教學使用。\n三、 本版分為課本、習作、附件、教師手冊(教學篇)、教師手冊(資料篇)、習作教師版六部分\n- 課本 課本由教學單元、數學好好玩、大家來解謎與素養實踐家所組成。「教學單元」的設計引導學生從觀察、操作、思考的過程中建立數學概念、培養數學技能 並增進數學溝通、表達與解決問題的能力。「數學好好玩」和「大家來解謎」的活動設計 讓學生在遊戲中學數學以及挑戰有趣的數學謎題。「素養實踐家」主要是讓學生統整及應用已學到的數學概念與技能 結合跨領域閱讀以解決綜合性數學問題。\n- 習作 習作配合各教學單元編寫 供學生上課或課後練習用。\n- 附件 配合課本活動設計、透過操作輔助學生建立數學概念。\n- 教師手冊(教學篇) 包含「本頁目標」、「教學準備」、「教學流程」、「教學建議」、「可能的解題策略」、「Q&A時間」及「類似題」等 並有課本解答頁面讓教師對照教學。並提供「單元學前檢測卷」複習已習得之先備經驗 並透過評量的方式確認學生對於上一階段的學習是否完備 以及「單元學後檢測卷」做單元學習後綜合性的評量 用以檢視學生本單元的學習成效。\n- 教師手冊(資料篇) 闡述說明本教材之編輯理念及具體實踐方法 並提供學習地圖、教材分布、課程綱要內容對照表等資料 另各單元均包含「教學計畫」、「教學研究」、「教材地位」、「知識脈絡」、「教學指導計畫」、「單元評量參考」、「連結架構」、「數學萬花筒」、「素養學習單」 協助教師掌握各單元的課程架構和相關教學研究資料的補充 方便教師教學參考。\n- 習作教師版 提供習作頁面的解答、評量目標 方便教師批改習作時參考。\n四、本書可搭配康軒線上媒體盒或康軒雲等與多媒體教學資源使用。\n- 提供各領域資源、教材製作、試卷服務、學生學習資源等。\n- 多媒體教學資源以動畫呈現教材內容 並提供配合各單元內容的學習單 期望以豐富的教學資源減輕教師負擔 使站在教學最前線的教師 以最有效、最方便的資源 輕鬆實現資訊融入各領域教學的新主張。\n---

In [5]:
# Convert Text Chunks into Embeddings (dense vector representation of the text that capture semantic information)

# Initialize the embedding model using Model on HuggingFace
# embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
embeddings = CohereEmbeddings(cohere_api_key="dBKmEAHnk6iFoBI26VsZGYR56lJ638CmzZcOUJeg")

# Initialize FAISS (Facebook AI Similarity Search) vector store, converting raw text chunks into embeddings
faiss_store = FAISS.from_texts(texts, embeddings)

# Define the save path and the name for the vector store
os.makedirs(vector_path, exist_ok=True)

# Save FAISS vector store to disk with a name
faiss_store.save_local(vector_path)